<a href="https://colab.research.google.com/github/canfielder/DSBA-6190_Proj2/blob/master/wine_predict/wine_quality_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [0]:
import pandas as pd
import numpy as np
import seaborn as sns

# Data Preparation
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

# Modeling
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

# Output
from sklearn.metrics import mean_squared_error,r2_score

# Data Import